In [1]:
!pip install tensorflow_decision_forests
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.8 MB 26.1 MB/s 
     |████████████████████████████████| 511.7 MB 5.6 kB/s 
     |████████████████████████████████| 1.6 MB 53.3 MB/s 
     |████████████████████████████████| 5.8 MB 8.9 MB/s 
     |████████████████████████████████| 438 kB 73.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Unins

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow_decision_forests as tfdf
import matplotlib.pyplot as plt

In [4]:
df = pd.read_csv("combined_data.csv",names=['text','label'])
print(df.head(5))

print(df.groupby('label').describe())
print("After removing Duplicates:")
df.drop_duplicates(subset='text',keep='first',inplace=True)
df.groupby('label').describe()


                                                text  label
0  #Depressed mood can be caused by infectious di...      0
1  With all of this unnessary  family drama, I fe...      1
2  I'm going to keep banging on about this, cos i...      0
3  Ugh. Woke up to the same acne covered face thi...      0
4  #stoner #instahookah #depressed Portable Trans...      0
        text                                                               
       count unique                                                top freq
label                                                                      
0      11470   9985    haIZ....frOM laSt wEek i keEP on goiNg out.....    6
1       5709   4373  Mom's depression tied to kids' emotional, inte...   11
After removing Duplicates:


text                                                               
      count unique                                                top freq
label                                                                     
0      9985   9985  #Depressed mood can be caused by infectious di...    1
1      4371   4371  With all of this unnessary  family drama, I fe...    1

In [5]:
df_positive=df[df['label']==1]
df_negative=df[df['label']==0]
df_negative_down = df_negative.sample(df_positive.shape[0])
df_negative_down.shape

df_balanced = pd.concat([df_positive,df_negative_down])
df_balanced

,text,label
1,"With all of this unnessary family drama, I fe...",1
6,"I feel so sad because i can't be happy, and th...",1
7,I went to church with my mom and a huge chunk ...,1
8,No love here.... #lonely #depressed pic.twitt...,1
9,I see so many people moving forward with their...,1
...,...,...
4827,26 thinking of moving family upset Okay so t...,0
4702,Should I meet estranged grandfather? I've nev...,0
12070,"Very excited about T4 out today, no more Arnie...",0
258,Deserve the world and if you feel like nobody ...,0


In [6]:
test_df = df_balanced.sample(frac=0.3, random_state=42)
train_df = df_balanced.drop(test_df.index)
print(f"Using {len(train_df)} samples for training and {len(test_df)} for validation")

Using 6119 samples for training and 2623 for validation


In [7]:
print("Training set:")
print(train_df["label"].value_counts())
print("Testing set:")
print(test_df["label"].value_counts())

Training set:
0    3077
1    3042
Name: label, dtype: int64
Testing set:
1    1329
0    1294
Name: label, dtype: int64


In [8]:
def create_dataset(dataframe):
    dataset = tf.data.Dataset.from_tensor_slices(
        (dataframe["text"].to_numpy(), dataframe["label"].to_numpy())
    )
    dataset = dataset.batch(100)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


train_ds = create_dataset(train_df)
test_ds = create_dataset(test_df)

In [9]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",trainable=True)


In [10]:
inputs = tf.keras.layers.Input(shape=(), dtype=tf.string)
tokenized = bert_preprocess(inputs)
outputs = bert_encoder(tokenized)['pooled_output']

preprocessor = keras.Model(inputs=inputs, outputs=outputs)
model = tfdf.keras.GradientBoostedTreesModel(preprocessing=preprocessor)

model.compile(metrics=["Accuracy", "Recall", "Precision", "AUC"])


Use /tmp/tmptr3snns4 as temporary training directory


In [11]:
model.fit(train_ds)

Reading training dataset...
Training dataset read in 0:01:23.136290. Found 6119 examples.
Training model...
Model trained in 0:03:39.292763
Compiling model...
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Model compiled.


In [12]:
model.summary()


Streaming output truncated to the last 5000 lines.
	model/keras_layer_1/StatefulPartitionedCall:13.66
	model/keras_layer_1/StatefulPartitionedCall:13.660
	model/keras_layer_1/StatefulPartitionedCall:13.661
	model/keras_layer_1/StatefulPartitionedCall:13.662
	model/keras_layer_1/StatefulPartitionedCall:13.663
	model/keras_layer_1/StatefulPartitionedCall:13.664
	model/keras_layer_1/StatefulPartitionedCall:13.665
	model/keras_layer_1/StatefulPartitionedCall:13.666
	model/keras_layer_1/StatefulPartitionedCall:13.667
	model/keras_layer_1/StatefulPartitionedCall:13.668
	model/keras_layer_1/StatefulPartitionedCall:13.669
	model/keras_layer_1/StatefulPartitionedCall:13.67
	model/keras_layer_1/StatefulPartitionedCall:13.670
	model/keras_layer_1/StatefulPartitionedCall:13.671
	model/keras_layer_1/StatefulPartitionedCall:13.672
	model/keras_layer_1/StatefulPartitionedCall:13.673
	model/keras_layer_1/StatefulPartitionedCall:13.674
	model/keras_layer_1/StatefulPartitionedCall:13.675
	model/keras_la

In [13]:
results = model.evaluate(test_ds, return_dict=True, verbose=0)
for name, value in results.items():
    print(f"{name}: {value:.4f}")

loss: 0.0000
Accuracy: 0.8818
recall: 0.8864
precision: 0.8811
auc: 0.9485


In [14]:
model.save('/content/',save_format='tf')

INFO:tensorflow:Assets written to: /content/assets


INFO:tensorflow:Assets written to: /content/assets
